<a href="https://colab.research.google.com/github/Saigeetha19112004/develop_a_script/blob/main/Beyondchats_Assisgnment_Round_by_Saigeetha_Balaji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Your task is to develop a script that does the following:
1. Takes as input a reddit user’s profile URL.
Example:
https://www.reddit.com/user/kojied/
https://www.reddit.com/user/Hungry-Move-6603/
2. Scrapes comments and posts created by the redditor.
3. Builds a User Persona based on details found on their reddit.
Click this image link to understand what a typical user persona looks like.
4. Output the user persona for the input profile in a text file.
5. For each characteristic in the user persona, the script also “cites” the posts or
comments it used to extract the specific user persona information.

#Install All the Libraries
- nltk
- spacy

In [1]:
%pip install praw requests beautifulsoup4 nltk spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 5.6 MB/s eta 0:00:00


In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 100.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Scrape reddit data




In [33]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import spacy
import os

#client ID of Saigeetha Balaji
#client Secret of Saigeetha Balaji
os.environ['REDDIT_CLIENT_ID'] = 'Cyj3sM1XWRMYfo5daKB1NQ'
os.environ['REDDIT_CLIENT_SECRET'] = 'FsquJTeQLvNzZtkhzPk2gKDTPRrY9A'

#other username for testing
username_to_analyze = "spez"
print(f"Scraping data for user: {username_to_analyze}")
comments, posts = scrape_reddit_user_data(username_to_analyze, num_comments=200, num_posts=200) # Scrape more data
print(f"Scraped {len(comments)} comments and {len(posts)} posts.")

# Check if scraping was successful
if not comments and not posts:
    print("Scraping failed or returned no data. Cannot proceed with processing.")
else:
    # Combine text from comments and posts
    all_text = ""
    for comment in comments:
        if comment and 'body' in comment and comment['body']:
            all_text += comment['body'] + " "

    for post in posts:
        # Use title and body for posts
        if post:
            if 'title' in post and post['title']:
                all_text += post['title'] + " "
            if 'body' in post and post['body']:
                all_text += post['body'] + " "

    # Basic NLP Preprocessing
    # Download necessary NLTK data if not already present
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        print("NLTK 'stopwords' resource not found. Downloading...")
        nltk.download('stopwords', quiet=True)

    # Load spaCy model
    try:
        nlp = spacy.load("en_core_web_sm")
    except OSError:
        print("SpaCy model 'en_core_web_sm' not found. Please run '!python -m spacy download en_core_web_sm' in a previous cell.")
        nlp = None

    processed_tokens = []
    if nlp:
        doc = nlp(all_text.lower())
        stop_words = set(stopwords.words('english'))
        processed_tokens = [token.text for token in doc if token.text.isalpha() and token.text not in stop_words]


    print(f"Original text length: {len(all_text)}")
    print(f"Number of processed tokens: {len(processed_tokens)}")


    # Create a DataFrame for easier handling of scraped data
    comments_df = pd.DataFrame(comments)
    posts_df = pd.DataFrame(posts)

    # Display the first few rows of the dataframes to verify
    print("Comments DataFrame:")
    display(comments_df.head())

    print("\nPosts DataFrame:")
    display(posts_df.head())

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Scraping data for user: spez


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Scraped 63 comments and 12 posts.
Original text length: 43577
Number of processed tokens: 3675
Comments DataFrame:


,id,body,subreddit,created_utc,url
0,mzyss3f,There's a message for all of us in this one.,Snoo,1.750977e+09,https://www.reddit.com/r/Snoo/comments/1ll8ago...
1,mzqqmaw,TFW you open Reddit at 3am.,Snoo,1.750876e+09,https://www.reddit.com/r/Snoo/comments/1ljn3d7...
2,mzk0y62,"It's cute, but cell phones are not permitted i...",Snoo,1.750789e+09,https://www.reddit.com/r/Snoo/comments/1lixlvp...
3,mzdhh2m,I am prepared to see some truly blursed Snoos.,Snoo,1.750703e+09,https://www.reddit.com/r/Snoo/comments/1liliw4...
4,mx2f9ja,Noted.,AskReddit,1.749582e+09,https://www.reddit.com/r/AskReddit/comments/1l...



Posts DataFrame:


,id,title,body,subreddit,created_utc,url,permalink
0,1lq0zjs,Best Snoo I could do.,[I forgot its ears again.](https://preview.red...,Snoo,1.751475e+09,https://www.reddit.com/r/Snoo/comments/1lq0zjs...,https://www.reddit.com/r/Snoo/comments/1lq0zjs...
1,1kfciml,"Reddit’s next chapter: smarter, easier, still ...","Hi everyone,\n\nI haven’t posted in a while—an...",u_spez,1.746454e+09,https://www.reddit.com/r/u_spez/comments/1kfci...,https://www.reddit.com/r/u_spez/comments/1kfci...
2,ipiung,Today we’re testing a new way to discuss polit...,,modnews,1.599668e+09,https://www.reddit.com/r/announcements/comment...,https://www.reddit.com/r/modnews/comments/ipiu...
3,6auyq9,Reddit is ProCSS,"Hi Mods,\n\nI wanted to follow up on the [CSS ...",modnews,1.494634e+09,https://www.reddit.com/r/modnews/comments/6auy...,https://www.reddit.com/r/modnews/comments/6auy...
4,693zcq,I hear the feedback on CSS / styles. I’m trave...,"Hey All,\n\nI just wanted to leave a quick not...",ModSupport,1.493854e+09,https://www.reddit.com/r/ModSupport/comments/6...,https://www.reddit.com/r/ModSupport/comments/6...


## Analyze scraped data





In [32]:
from collections import Counter
from nltk.util import bigrams
import spacy

# Analyze word frequency
word_counts = Counter(processed_tokens)
most_common_words = word_counts.most_common(50) # Get the 50 most common words

print("Most Common Words:")
for word, count in most_common_words:
    print(f"{word}: {count}")

# Analyze bigram frequency (pairs of words)
# Create bigrams from the processed tokens
bigram_list = list(bigrams(processed_tokens))
bigram_counts = Counter(bigram_list)
most_common_bigrams = bigram_counts.most_common(30) # Get the 30 most common bigrams

print("\nMost Common Bigrams:")
for bigram, count in most_common_bigrams:
    print(f"{bigram[0]} {bigram[1]}: {count}")


Most Common Words:
reddit: 83
r: 45
community: 37
communities: 35
time: 27
us: 25
make: 25
like: 23
new: 23
ads: 22
want: 21
know: 21
first: 19
better: 18
way: 18
many: 18
would: 18
political: 17
think: 16
last: 16
one: 15
best: 15
today: 15
things: 15
css: 15
years: 15
posts: 15
see: 14
feedback: 14
working: 14
mods: 14
announcements: 14
lot: 13
people: 13
also: 13
team: 13
discussion: 13
users: 13
tools: 13
api: 13
post: 12
back: 12
next: 11
subreddits: 11
going: 11
approach: 11
work: 11
help: 11
subreddit: 10
even: 10

Most Common Bigrams:
r announcements: 13
political ads: 9
reddit answers: 5
mod tools: 5
make sure: 4
years ago: 4
status quo: 4
third party: 4
community team: 4
things like: 3
want know: 3
jelly belly: 3
last year: 3
reddit community: 3
reddit grown: 3
discussion posts: 3
announcements comments: 3
many communities: 3
communities want: 3
ebook dp: 3
new system: 3
widget system: 3
r modsupport: 3
trust safety: 3
api terms: 3
better better: 2
long run: 2
single culture:

## Generate user persona



In [34]:
#generating my own persona
persona = {
    "Key Interests": [],
    "Common Topics": [],
    "Tone": "Based on the scraped data, the tone appears to be primarily related to discussions about Reddit itself, its communities, tools, and policies.", # Placeholder, more sophisticated analysis needed for nuanced tone
    "Summary": ""
}
#filters
generic_terms = set(['reddit', 'community', 'communities', 'like', 'make', 'time', 'us', 'know', 'want', 'first', 'better', 'way', 'many', 'would', 'think', 'last', 'one', 'best', 'today', 'things', 'css', 'years', 'posts', 'see', 'feedback', 'working', 'mods', 'announcements', 'lot', 'people', 'also', 'team', 'discussion', 'users', 'tools', 'api', 'post', 'back', 'next', 'subreddits', 'going', 'approach', 'work', 'help', 'subreddit', 'even', 'r'])

# Add key interests based on most common words
for word, count in most_common_words:
    if word not in generic_terms:
        persona["Key Interests"].append(word.capitalize()) # Capitalize for persona

# Add common topics based on most common bigrams
for bigram, count in most_common_bigrams:
     # Join bigram words and capitalize the first word
    bigram_phrase = f"{bigram[0].capitalize()} {bigram[1]}"
    persona["Common Topics"].append(bigram_phrase)

# Generate a summary
summary_points = []
if persona["Key Interests"]:
    summary_points.append(f"The user shows interest in: {', '.join(persona['Key Interests'][:5])}.") # Limit to top 5 interests
if persona["Common Topics"]:
    summary_points.append(f"Frequent topics include: {', '.join(persona['Common Topics'][:5])}.") # Limit to top 5 topics

persona["Summary"] = " ".join(summary_points) + " " + persona["Tone"]


print("Generated User Persona (Draft):")
import json
print(json.dumps(persona, indent=4))


citations = {
    "Key Interests": "Derived from frequent terms in comments and posts.",
    "Common Topics": "Derived from frequent word pairs (bigrams) in comments and posts.",
    "Tone": "General observation based on content.",
    "Source Data": {
        "Comments": "Data scraped from user comments.",
        "Posts": "Data scraped from user posts (submissions)."
    }
}

print("\nCitations (Basic):")
print(json.dumps(citations, indent=4))

Generated User Persona (Draft):
{
    "Key Interests": [
        "New",
        "Ads",
        "Political"
    ],
    "Common Topics": [
        "R announcements",
        "Political ads",
        "Reddit answers",
        "Mod tools",
        "Make sure",
        "Years ago",
        "Status quo",
        "Third party",
        "Community team",
        "Things like",
        "Want know",
        "Jelly belly",
        "Last year",
        "Reddit community",
        "Reddit grown",
        "Discussion posts",
        "Announcements comments",
        "Many communities",
        "Communities want",
        "Ebook dp",
        "New system",
        "Widget system",
        "R modsupport",
        "Trust safety",
        "Api terms",
        "Better better",
        "Long run",
        "Single culture",
        "Reddit today",
        "See people"
    ],
    "Tone": "Based on the scraped data, the tone appears to be primarily related to discussions about Reddit itself, its communities, 

## Output to text file



In [35]:
#Output :Text file

# Define the filename
filename = 'user_persona_with_citations.txt'

# Format the user persona and citations for the text file
output_content = f"# Reddit User Persona\n\n"

# Add Persona Details
output_content += "## Persona Details\n"
for key, value in persona.items():
    output_content += f"**{key}:**\n"
    if isinstance(value, list):
        for item in value:
            output_content += f"- {item}\n"
    else:
        output_content += f"{value}\n"
    output_content += "\n" # Add a blank line after each section

# Add Citations
output_content += "## Citations\n"
for key, value in citations.items():
    if isinstance(value, dict):
        output_content += f"**{key}:**\n"
        for sub_key, sub_value in value.items():
            output_content += f"- {sub_key}: {sub_value}\n"
    else:
        output_content += f"**{key}:** {value}\n"
    output_content += "\n" # Add a blank line after each citation section


# Write the content to the text file
try:
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(output_content)
    print(f"User persona with citations successfully written to {filename}")
except IOError as e:
    print(f"Error writing to file {filename}: {e}")

User persona with citations successfully written to user_persona_with_citations.txt
